# Example

In [1]:
# can be removed once work on underlying algorithms is done
%load_ext autoreload
%autoreload 2

In [2]:
# needs to be removed for public version, replace by note that setting proxy may be necessary
%env http_proxy=139.7.95.77:8080
%env https_proxy=139.7.95.77:8080

env: http_proxy=139.7.95.77:8080
env: https_proxy=139.7.95.77:8080


## Imports

In [2]:
import random

import networkx as nx

import osmnx as ox
import pyproj

from imports import networkx as imp_nx
from network import Steiner as stp

ModuleNotFoundError: No module named 'osmnx'

In [ ]:
# needs to be removed later...
import sys
sys.path.append("/home/rbardeli/code/graphilp/b_thomas_1/")

## Load streetmap graph

In [ ]:
crs = pyproj.crs.CRS('epsg:31467')

In [ ]:
#place = 'Maxvorstadt, München, Deutschland'
place = 'Oberkassel, Düsseldorf, Deutschland'

# road network of suburb (converted to Gauss-Krüger 3)
G = ox.project_graph(ox.graph_from_place(place, network_type='walk'), to_crs=crs)

In [ ]:
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color='b');

# Choose terminals

In [ ]:
node_list = list(G.nodes())

In [ ]:
terminals = [node_list[random.randint(0, len(node_list))] for n in range(5)]

In [ ]:
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color=['#ED0000' if n in terminals else '#00F' for n in node_list],
    node_size=[50 if n in terminals else 15 for n in node_list]);

## Set up and run optimisation problem

In [ ]:
# Transform input into an undirected graph
G2 = nx.Graph(G)

In [ ]:
# Import to GraphILP API
optG = imp_nx.read(G2)

In [ ]:
m = stp.createModel(optG, terminals, weight='length')

In [ ]:
m.optimize(callback=stp.callback_cycle)

## Visualise solution

In [ ]:
solution = stp.extractSolution(optG, m)

In [ ]:
edge_colors = ['#ED0000' if (u,v) in solution or (v, u) in solution else '#AAA' for u, v in G.edges()]
edge_widths = [3 if (u,v) in solution else 1 for u, v in G.edges()]

In [ ]:
ox.plot_graph(G, figsize=(14, 10),
    bgcolor='#FFF',
    node_color=['#ED0000' if n in terminals else '#00F' for n in node_list],
    node_size=[75 if n in terminals else 15 for n in node_list],
             edge_color=edge_colors, edge_linewidth=edge_widths);